In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, date

---
#### To-Do List 
* City:
    * Hawaii
---
* Step To Do:
    1. Clean Data
    2. Reformat data
    3. Data Analysis
---
* Analysis
    * Room type Analysis Overall
    * Property Type Analysis Overall
    * % of Fake Superhost over All Superhost Each Main Property Type
    * list of Fake Superhost Each Main Property Type

In [2]:
listing = pd.read_csv('listingsSep.csv')
listing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22566 entries, 0 to 22565
Data columns (total 74 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            22566 non-null  int64  
 1   listing_url                                   22566 non-null  object 
 2   scrape_id                                     22566 non-null  float64
 3   last_scraped                                  22566 non-null  object 
 4   name                                          22566 non-null  object 
 5   description                                   22388 non-null  object 
 6   neighborhood_overview                         12651 non-null  object 
 7   picture_url                                   22566 non-null  object 
 8   host_id                                       22566 non-null  int64  
 9   host_url                                      22566 non-null 

In [3]:
listing2020 = pd.read_csv('listingsJan.csv')
listing2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21876 entries, 0 to 21875
Data columns (total 74 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            21876 non-null  int64  
 1   listing_url                                   21876 non-null  object 
 2   scrape_id                                     21876 non-null  int64  
 3   last_scraped                                  21876 non-null  object 
 4   name                                          21876 non-null  object 
 5   description                                   21581 non-null  object 
 6   neighborhood_overview                         12730 non-null  object 
 7   picture_url                                   21876 non-null  object 
 8   host_id                                       21876 non-null  int64  
 9   host_url                                      21876 non-null 

In [4]:
review2020 = pd.read_csv('reviews2020.csv')
review2021 = pd.read_csv('reviews.csv')
review2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 552300 entries, 0 to 552299
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   listing_id  552300 non-null  int64 
 1   date        552300 non-null  object
dtypes: int64(1), object(1)
memory usage: 8.4+ MB


In [5]:
review2021 = review2021.merge(listing, left_on='listing_id', right_on='id')[['listing_id','date']]
review2021

,listing_id,date
0,5065,2013-02-18
1,5065,2013-05-03
2,5065,2015-11-29
3,5065,2015-12-27
4,5065,2016-01-05
...,...,...
651459,51914042,2021-09-12
651460,51920037,2021-09-09
651461,51938777,2021-09-03
651462,51938777,2021-09-06


In [6]:
listing['number_of_reviews'].sum()

651471

In [7]:
review2020 = review2020.merge(listing2020, left_on='listing_id', right_on='id')[['listing_id','date']]
review2020

,listing_id,date
0,5269,2011-05-31
1,5269,2011-11-25
2,5269,2018-11-28
3,5269,2019-07-23
4,5269,2019-09-04
...,...,...
552295,47470360,2021-01-13
552296,47497230,2021-01-08
552297,47497514,2021-01-14
552298,47514916,2021-01-10


In [8]:
listing2020['number_of_reviews'].sum()

552300

# Step 1: Clean Data 

### Drop Columns

In [9]:
# Drop columns
listing = listing.drop(listing.iloc[:,61:68],axis=1)
listing2020 = listing2020.drop(listing2020.iloc[:,61:68],axis=1)

# Drop more than 50% of Null in each column
drop_list = listing.isnull().sum() / len(listing)
more_50 = listing.columns[drop_list > 0.5]
listing = listing.drop(more_50, axis = 1)

drop_list2020 = listing2020.isnull().sum() / len(listing)
more_50_2020 = listing2020.columns[drop_list2020 > 0.5]
listing2020 = listing2020.drop(more_50_2020, axis = 1)



# Drop Others columns
listing = listing.drop(['scrape_id','description','neighborhood_overview','host_location',
                        'host_verifications','number_of_reviews_ltm','has_availability',
                        'name','host_about','last_scraped','host_neighbourhood',
                        'host_acceptance_rate'], axis=1)

listing2020 = listing2020.drop(['scrape_id','description','neighborhood_overview','host_location',
                        'host_verifications','number_of_reviews_ltm','has_availability',
                        'name','host_about','last_scraped','host_neighbourhood',
                        'host_acceptance_rate'], axis=1)


#listing.info()

# Step 2: Reformat Data 

### Drop Row and Reformat Column Value

In [10]:
# Erase % to ''
chg_rate = listing.columns.str.contains('rate')
rate = listing.columns[chg_rate == True]
listing[rate] = listing[rate].replace('%','', regex = True)

# Erase $ to '' & change data type in price
listing['price'] = listing['price'].str.replace('$','', regex = True)
listing['price'] = listing['price'].str.replace(',','', regex = True).astype(float)

# Convert string to datetime
listing['last_review'] = pd.to_datetime(listing['last_review'])
listing['first_review'] = pd.to_datetime(listing['first_review'])

# Drop 0 
listing['host_response_rate']   = listing['host_response_rate'].fillna(0)
listing = listing.loc[listing['host_response_rate'] != 0]
listing['host_response_rate']   = listing['host_response_rate'].astype(int)
listing['review_scores_rating'] = listing['review_scores_rating'].fillna(0.0)
listing = listing.loc[listing['review_scores_rating'] != 0.0]
listing['number_of_reviews']    = listing['number_of_reviews'].fillna(0)
listing = listing.loc[listing['last_review'] != np.nan]
listing = listing.loc[listing['price'] != 0]


In [11]:
# Match listing_id with our listing data frame
review2021 = review2021.merge(listing, left_on='listing_id', right_on='id')[['listing_id','date']]
review2021

,listing_id,date
0,5065,2013-02-18
1,5065,2013-05-03
2,5065,2015-11-29
3,5065,2015-12-27
4,5065,2016-01-05
...,...,...
585644,51914042,2021-09-12
585645,51920037,2021-09-09
585646,51938777,2021-09-03
585647,51938777,2021-09-06


In [12]:
# Erase % to ''
chg_rate2020 = listing2020.columns.str.contains('rate')
rate2020 = listing2020.columns[chg_rate2020 == True]
listing2020[rate2020] = listing2020[rate2020].replace('%','', regex = True)

# Erase $ to '' & change data type in price
listing2020['price'] = listing2020['price'].str.replace('$','', regex = True)
listing2020['price'] = listing2020['price'].str.replace(',','', regex = True).astype(float)

# Convert string to datetime
listing2020['last_review'] = pd.to_datetime(listing2020['last_review'])
listing2020['first_review'] = pd.to_datetime(listing2020['first_review'])

# Drop 0 
listing2020['host_response_rate']   = listing2020['host_response_rate'].fillna(0)
listing2020 = listing2020.loc[listing2020['host_response_rate'] != 0]
listing2020['host_response_rate']   = listing2020['host_response_rate'].astype(int)
listing2020['review_scores_rating'] = listing2020['review_scores_rating'].fillna(0.0)
listing2020 = listing2020.loc[listing2020['review_scores_rating'] != 0.0]
listing2020['number_of_reviews']    = listing2020['number_of_reviews'].fillna(0)
listing2020 = listing2020.loc[listing2020['last_review'] != np.nan]

review2020 = review2020.merge(listing2020, left_on='listing_id', right_on='id')[['listing_id','date']]
review2020

,listing_id,date
0,5269,2011-05-31
1,5269,2011-11-25
2,5269,2018-11-28
3,5269,2019-07-23
4,5269,2019-09-04
...,...,...
513838,47470360,2021-01-13
513839,47497230,2021-01-08
513840,47497514,2021-01-14
513841,47514916,2021-01-10


In [13]:
review2020['date'] = pd.to_datetime(review2020['date'])
review2021['date'] = pd.to_datetime(review2021['date'])

In [14]:
listing['last_review'].isnull().any()

False

In [15]:
listing2020['last_review'].isnull().any()

False

In [16]:
print(listing['last_review'],listing2020['last_review'])

0       2020-03-22
1       2021-08-16
2       2021-09-05
3       2021-06-26
4       2021-08-25
           ...    
22364   2021-09-07
22366   2021-09-12
22368   2021-09-09
22375   2021-09-06
22386   2021-09-03
Name: last_review, Length: 16135, dtype: datetime64[ns] 0       2021-01-02
1       2020-03-18
2       2020-12-19
3       2020-12-12
4       2018-05-11
           ...    
21696   2021-01-13
21750   2021-01-08
21751   2021-01-14
21773   2021-01-10
21828   2021-01-12
Name: last_review, Length: 14894, dtype: datetime64[ns]


In [17]:
print(pd.unique(listing['room_type']))

['Entire home/apt' 'Private room' 'Hotel room' 'Shared room']


In [18]:
print(pd.unique(listing2020['property_type']))

['Entire apartment' 'Entire house' 'Entire condominium'
 'Entire guest suite' 'Private room in guest suite' 'Farm stay'
 'Entire bungalow' 'Private room in house' 'Room in boutique hotel'
 'Private room in bed and breakfast' 'Room in hotel' 'Entire villa'
 'Private room in resort' 'Entire cottage' 'Private room in apartment'
 'Entire guesthouse' 'Entire townhouse' 'Entire cabin'
 'Private room in guesthouse' 'Private room in farm stay' 'Tent'
 'Private room in condominium' 'Shared room in hostel'
 'Room in bed and breakfast' 'Private room in cottage'
 'Private room in treehouse' 'Hut' 'Earth house' 'Entire place'
 'Private room in nature lodge' 'Private room in cabin' 'Tiny house'
 'Entire serviced apartment' 'Entire bed and breakfast' 'Private room'
 'Room in aparthotel' 'Island' 'Yurt' 'Room in resort' 'Entire loft'
 'Boat' 'Private room in townhouse' 'Private room in hostel'
 'Shared room in house' 'Private room in bungalow' 'Private room in hut'
 'Room in nature lodge' 'Treehouse' 

# Data Analysis

---
### Overall Room_type 

In [19]:
room_overall = listing[['id','host_name','host_response_rate','review_scores_rating','number_of_reviews',
                  'host_is_superhost','last_review','property_type','room_type','price']]
# Numbers listings of Each Room Type
c = room_overall.groupby('room_type').size().reset_index(name='counts')
c['total'] = c['counts'].sum()
c['percent(%)'] = round((c['counts'] / c['counts'].sum()) * 100 , 2)
c

,room_type,counts,total,percent(%)
0,Entire home/apt,14675,16135,90.95
1,Hotel room,74,16135,0.46
2,Private room,1358,16135,8.42
3,Shared room,28,16135,0.17


In [20]:
# % of price different
p = room_overall.groupby('room_type').agg({'price':'sum'}).reset_index()
p['percent(%)'] = round((p['price'] / p['price'].sum()) * 100 , 2)
p

,room_type,price,percent(%)
0,Entire home/apt,4500094.0,94.02
1,Hotel room,23936.0,0.50
2,Private room,260712.0,5.45
3,Shared room,1779.0,0.04


In [21]:
# Excel
c.to_excel('Overall_Room_type.xlsx')

---
### Overall Entire home/apt Property

In [22]:
entire2 = listing[['id','host_name','host_response_rate','review_scores_rating','number_of_reviews',
                  'host_is_superhost','last_review','property_type','room_type','price']]
print(pd.unique(entire2['property_type']))

['Entire bed and breakfast' 'Entire rental unit' 'Entire residential home'
 'Entire condominium (condo)' 'Entire guest suite' 'Farm stay'
 'Entire bungalow' 'Private room in residential home'
 'Private room in bed and breakfast' 'Room in hotel' 'Treehouse'
 'Entire villa' 'Entire cottage' 'Entire guesthouse' 'Entire townhouse'
 'Private room in resort' 'Room in boutique hotel'
 'Private room in rental unit' 'Entire cabin' 'Private room in guesthouse'
 'Room in bed and breakfast' 'Private room in farm stay' 'Tent'
 'Private room in condominium (condo)' 'Shared room in hostel' 'Hut'
 'Earth house' 'Entire place' 'Private room in nature lodge'
 'Private room in guest suite' 'Private room in cabin' 'Tiny house'
 'Entire serviced apartment' 'Private room' 'Yurt' 'Room in aparthotel'
 'Island' 'Entire loft' 'Private room in cottage' 'Room in resort'
 'Private room in townhouse' 'Private room in hostel'
 'Private room in hut' 'Room in nature lodge'
 'Private room in serviced apartment' 'Priva

In [23]:
# % count
e = entire2.groupby('property_type').size().reset_index(name='counts').sort_values(by=['counts'], ascending=False)
e['total'] = e['counts'].sum()
e['percent(%)'] = round((e['counts'] / e['total']) *100 , 2)
e.head(30)

,property_type,counts,total,percent(%)
9,Entire condominium (condo),8586,16135,53.21
17,Entire residential home,2029,16135,12.58
16,Entire rental unit,1732,16135,10.73
11,Entire guest suite,445,16135,2.76
44,Private room in residential home,417,16135,2.58
20,Entire townhouse,361,16135,2.24
12,Entire guesthouse,289,16135,1.79
22,Entire villa,249,16135,1.54
10,Entire cottage,240,16135,1.49
19,Entire serviced apartment,178,16135,1.10


In [24]:
#excel
e.to_excel('Overall_Property_Type.xlsx')

---
### SuperHost Requirement
* Completed at least 10 trips 
* Maintained a 90% response rate or higher
* Maintained a 1% cancellation rate (1 cancellation per 100 reservations) or lower, with exceptions made for those that fall under our Extenuating Circumstances policy
* Maintained a 4.8 overall rating (this rating looks at the past 365 days of reviews, based on the date the guest left a review, not the date the guest checked out)

### Advantage to become a superhost
Superhosts earn a 100USD Airbnb coupon every year that they keep their status. And when they refer a new Host to sign up, Superhosts get an extra 20% on top of the usual referral bonus

https://www.airbnb.com/d/superhost

---

In [25]:
host_listing7 = listing[['id','host_id','host_name','host_response_rate','review_scores_rating','number_of_reviews',
                        'host_is_superhost','last_review','property_type','room_type','price',
                        'neighbourhood_cleansed','neighbourhood_group_cleansed','latitude','longitude','listing_url']]
host_listing7['overall_rating'] = round(host_listing7.groupby(['host_id','host_is_superhost'])['review_scores_rating'].transform('mean'),2)
host_listing7['Host_List_count'] = host_listing7.groupby('host_id')['host_id'].transform('count')
host_listing7['Avg_Price'] =round( host_listing7.groupby('host_id')['price'].transform('mean'),2)
host_listing7


<ipython-input-25-de27de36cf2f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  host_listing7['overall_rating'] = round(host_listing7.groupby(['host_id','host_is_superhost'])['review_scores_rating'].transform('mean'),2)
<ipython-input-25-de27de36cf2f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  host_listing7['Host_List_count'] = host_listing7.groupby('host_id')['host_id'].transform('count')
<ipython-input-25-de27de36cf2f>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

,id,host_id,host_name,host_response_rate,review_scores_rating,number_of_reviews,host_is_superhost,last_review,property_type,room_type,price,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,listing_url,overall_rating,Host_List_count,Avg_Price
0,5065,7257,Wayne,100,4.60,42,f,2020-03-22,Entire bed and breakfast,Entire home/apt,85.0,Hamakua,Hawaii,20.04266,-155.43259,https://www.airbnb.com/rooms/5065,4.60,1,85.0
1,5269,7620,Lea & Pat,100,4.46,14,t,2021-08-16,Entire rental unit,Entire home/apt,149.0,South Kohala,Hawaii,20.02740,-155.70200,https://www.airbnb.com/rooms/5269,4.55,2,124.5
2,5387,7878,Edward,100,4.64,181,f,2021-09-05,Entire rental unit,Entire home/apt,85.0,South Kona,Hawaii,19.43119,-155.88079,https://www.airbnb.com/rooms/5387,4.63,3,149.0
3,5389,7878,Edward,100,4.83,31,f,2021-06-26,Entire residential home,Entire home/apt,264.0,North Kona,Hawaii,19.56413,-155.96347,https://www.airbnb.com/rooms/5389,4.63,3,149.0
4,5390,7887,Todd,100,4.41,154,f,2021-08-25,Entire condominium (condo),Entire home/apt,98.0,Koloa-Poipu,Kauai,21.88151,-159.47346,https://www.airbnb.com/rooms/5390,4.41,1,98.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22364,51911263,371942068,Joy,100,5.00,1,f,2021-09-07,Entire condominium (condo),Entire home/apt,209.0,Kapaa-Wailua,Kauai,22.05356,-159.32604,https://www.airbnb.com/rooms/51911263,4.73,30,260.0
22366,51914042,122959159,Jason,100,4.00,1,t,2021-09-12,Entire rental unit,Entire home/apt,103.0,North Kona,Hawaii,19.72288,-155.98838,https://www.airbnb.com/rooms/51914042,4.45,2,96.5
22368,51920037,9531956,Andrea,100,5.00,1,f,2021-09-09,Private room in tiny house,Private room,38.0,Puna,Hawaii,19.61827,-154.94679,https://www.airbnb.com/rooms/51920037,5.00,1,38.0
22375,51938777,132551861,Liberty,100,4.00,2,f,2021-09-06,Private room in residential home,Private room,83.0,Paia-Haiku,Maui,20.91665,-156.27145,https://www.airbnb.com/rooms/51938777,4.35,2,84.0


In [26]:
host_listing7[host_listing7['property_type']=='Bus']

,id,host_id,host_name,host_response_rate,review_scores_rating,number_of_reviews,host_is_superhost,last_review,property_type,room_type,price,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,listing_url,overall_rating,Host_List_count,Avg_Price
16754,46464517,77607646,Feagaiga,100,5.00,4,t,2021-01-11,Bus,Entire home/apt,20.0,Waianae,Honolulu,21.45561,-158.19623,https://www.airbnb.com/rooms/46464517,5.00,1,20.0
20218,52556286,232546534,Samuel,100,4.86,7,f,2021-08-12,Bus,Entire home/apt,225.0,Wailuku-Kahului,Maui,20.88074,-156.45554,https://www.airbnb.com/rooms/52556286,4.86,1,225.0


In [27]:
host_listing7.to_excel('Filtered_Host_Listing2.xlsx', sheet_name = 'Sheet1')

In [28]:
host_listing2020 = listing2020[['id','host_id','host_name','host_response_rate','review_scores_rating','number_of_reviews',
                        'host_is_superhost','last_review','property_type','room_type','price',
                        'neighbourhood_cleansed','neighbourhood_group_cleansed','latitude','longitude','listing_url']]
host_listing2020['overall_rating'] = round(host_listing2020.groupby(['host_id','host_is_superhost'])['review_scores_rating'].transform('mean'),2)
#host_listing2020[host_listing2020['host_id']==903292]

<ipython-input-28-6dc69df6849b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  host_listing2020['overall_rating'] = round(host_listing2020.groupby(['host_id','host_is_superhost'])['review_scores_rating'].transform('mean'),2)


In [29]:
# comparing number of review
host_listing2021 = host_listing7[['host_is_superhost','number_of_reviews']]
host_listing2021.groupby('host_is_superhost').agg({'number_of_reviews':'sum'}).reset_index()


,host_is_superhost,number_of_reviews
0,f,196569
1,t,389087


In [30]:
389087+196703

585790

---
### Avg Pricing SuperHost vs NormalHost
---

--- 
### Normal Host

In [31]:
host_index = host_listing7[(host_listing7['host_is_superhost'] == "f")].index
host_index

Int64Index([    0,     2,     3,     4,     8,    10,    14,    15,    16,
               17,
            ...
            22250, 22254, 22292, 22294, 22309, 22317, 22364, 22368, 22375,
            22386],
           dtype='int64', length=8662)

In [32]:
host = host_listing7.loc[host_index]
host = host.drop_duplicates(subset=['host_id'])
host.shape

(2606, 19)

---
### Fake SuperHost

In [33]:
# Finding incorrect host_is_superhost

index_fake = host_listing7[((host_listing7['host_response_rate'] < 90) | (host_listing7['review_scores_rating'] < 4.8))
                      & (host_listing7['host_is_superhost'] == "t")].index
index_fake

Int64Index([    1,     5,     7,    12,    13,    26,    27,    33,    34,
               41,
            ...
            21762, 21792, 21796, 21829, 21862, 21977, 22069, 22072, 22217,
            22366],
           dtype='int64', length=1805)

### True SuperHost

In [34]:
index_true = host_listing7[((host_listing7['host_response_rate'] >= 90) & (host_listing7['review_scores_rating'] >= 4.8))
                      & (host_listing7['host_is_superhost'] == "t")].index
index_true

Int64Index([   11,    19,    22,    23,    24,    32,    35,    37,    39,
               40,
            ...
            22095, 22103, 22117, 22129, 22140, 22167, 22175, 22325, 22334,
            22341],
           dtype='int64', length=5668)

In [35]:
# Get rip of same host with Avg rating < 4.8
listingT = host_listing7.loc[index_true]
listingTf = listingT[(listingT['overall_rating'] < 4.8)]
listingT = listingT[(listingT['overall_rating'] >= 4.8)]
# For lsting Top 50 Superhost
Truelisting = listingT
FalseList = listingTf


In [36]:
listingT = listingT.drop_duplicates(subset=['host_id'])
listingTf = listingTf.drop_duplicates(subset=['host_id'])
listingT.shape

(2453, 19)

### List of Inaccurated Superhost

In [37]:
listingf7 = host_listing7.loc[index_fake]
listingfT7 = listingf7
listingf7 = listingf7[(listingf7['overall_rating'] < 4.8)]
listingfT7 = listingfT7[(listingfT7['overall_rating'] >= 4.8)]
Truelisting2 = listingfT7
FalseList2 = listingf7
listingf7.shape

(1065, 19)

In [38]:
listingfT7 = listingfT7.drop_duplicates(subset=['host_id'])
listingf7 = listingf7.drop_duplicates(subset=['host_id'])
listingf7.shape

(454, 19)

In [39]:
# For Top50
dup_host = [Truelisting2,Truelisting]

In [40]:
#Add True SuperHost
frames = [listingfT7,listingT]
True_Superhost = pd.concat(frames)
True_Superhost.shape

(2754, 19)

In [41]:
#Add False SuperHost
frames2 = [listingTf,listingf7]
False_Superhost = pd.concat(frames2)
False_Superhost.shape


(599, 19)

In [42]:
#Excel
#fake_superhost['last_review'] = fake_superhost['last_review'].dt.date                       
False_Superhost.to_excel('Fake_superhost.xlsx',sheet_name='Sheet1')

---

### Normal Host vs Super Host

In [43]:
frame3 = [host,False_Superhost,True_Superhost]
allhost = pd.concat(frame3)
size = allhost[['host_is_superhost']]
size = size.groupby('host_is_superhost').size().reset_index(name='counts')
size['total'] = size['counts'].sum()
size['pecent(%)'] = round((size['counts']/size['counts'].sum())*100,2)
size


,host_is_superhost,counts,total,pecent(%)
0,f,2606,5959,43.73
1,t,3353,5959,56.27


In [44]:
size.to_excel('Pecentage_Of_Both_Host.xlsx',sheet_name='Sheet1')

---
### True SuperHost Vs Fake SuperHost

In [45]:
overfake = False_Superhost.groupby('host_is_superhost').size().reset_index(name='counts')
oversuper = True_Superhost.groupby('host_is_superhost').size().reset_index(name='counts') 

frames = [overfake,oversuper]
overall = pd.concat(frames)
overall = overall.reset_index()
overall = overall.rename(columns={'index': 'Condtion'})
overall.iloc[:1,:1] = 'FakeSuperhost'
overall.iloc[1:2,:1] = 'Superhost'
overall['total'] = overall['counts'].sum()
overall['percent(%)'] = round((overall['counts'] / overall['total']) *100 , 2)
overall['collect_date'] = 'Sep'
overall

,Condtion,host_is_superhost,counts,total,percent(%),collect_date
0,FakeSuperhost,t,599,3353,17.86,Sep
1,Superhost,t,2754,3353,82.14,Sep


In [46]:
#excel
overall.to_excel('Pecentage_Fake_superhost.xlsx')

---

### Timeline Comparison
#### 2020 vs 2021

In [47]:
date_2020 = review2020.loc[(review2020['date'] >='2020-01-01') & (review2020['date'] <='2020-12-31')]
date_2021 = review2021.loc[(review2021['date'] >='2021-01-01') & (review2021['date'] <='2021-09-30')]

In [48]:
date_time = pd.concat([date_2020,date_2021]).sort_values(by='date')
date_time.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226338 entries, 468365 to 580591
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   listing_id  226338 non-null  int64         
 1   date        226338 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1)
memory usage: 5.2 MB


In [49]:
date_time['Months'] = date_time['date'].dt.strftime('%B')
col1 = date_time.pop('Months')
date_time.insert(2, 'Months', col1)

date_time['Month'] = date_time['date'].dt.strftime('%m')
col2 = date_time.pop('Month')
date_time.insert(3, 'Month', col2)

date_time['Year'] = date_time['date'].dt.strftime('%y')
col3 = date_time.pop('Year')
date_time.insert(4, 'Year', col3)

date_time['Month/Year'] = date_time['date'].dt.strftime('%B,%y')
col4 = date_time.pop('Month/Year')
date_time.insert(5, 'Month/Year', col4)

date_time

,listing_id,date,Months,Month,Year,Month/Year
468365,32641774,2020-01-01,January,01,20,"January,20"
363917,21802690,2020-01-01,January,01,20,"January,20"
452741,30602884,2020-01-01,January,01,20,"January,20"
410983,25319302,2020-01-01,January,01,20,"January,20"
452802,30672460,2020-01-01,January,01,20,"January,20"
...,...,...,...,...,...,...
405753,23976506,2021-09-12,September,09,21,"September,21"
520746,37206966,2021-09-12,September,09,21,"September,21"
340482,20093438,2021-09-12,September,09,21,"September,21"
281906,16546487,2021-09-12,September,09,21,"September,21"


In [50]:
date_time.groupby('Month/Year').count()

,listing_id,date,Months,Month,Year
Month/Year,,,,,
"April,20",731,731,731,731,731
"April,21",16208,16208,16208,16208,16208
"August,20",2233,2233,2233,2233,2233
"August,21",24706,24706,24706,24706,24706
"December,20",10652,10652,10652,10652,10652
"February,20",17762,17762,17762,17762,17762
"February,21",9610,9610,9610,9610,9610
"January,20",19034,19034,19034,19034,19034
"January,21",9377,9377,9377,9377,9377


In [51]:
date_time.to_excel('date_diff.xlsx',sheet_name='Sheet1')

---
### Super host Cretiria


In [52]:
d = {'SuperHost Requirement': ['4.8+ Overall Rating ',
                               '90% Response Rate ',
                              '10+ Stays ',
            
                               '<1% Cancellation Rate'] , 
     'Before Covid 19': ['√','√','√','√'],
    'During Covid 19': ['√','√','X','X']}
cretiria = pd.DataFrame(data = d)
cretiria.to_excel('cretiria.xlsx',sheet_name='Sheet1')

In [53]:
cretiria

,SuperHost Requirement,Before Covid 19,During Covid 19
0,4.8+ Overall Rating,√,√
1,90% Response Rate,√,√
2,10+ Stays,√,X
3,<1% Cancellation Rate,√,X


---

In [54]:
df = host_listing7[['host_id','host_name','price']]
#Number_host_listing.duplicated(subset=['host_id']).sum()
#Number_host_listing = Number_host_listing[Number_host_listing.groupby('host_id').count()
#Number_host_listing.sort_values(by = 'host_name',ascending=False).head(50)
df['count'] = df.groupby('host_id')['host_id'].transform('count')
df

<ipython-input-54-8b2fb3fa1a24>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['count'] = df.groupby('host_id')['host_id'].transform('count')


,host_id,host_name,price,count
0,7257,Wayne,85.0,1
1,7620,Lea & Pat,149.0,2
2,7878,Edward,85.0,3
3,7878,Edward,264.0,3
4,7887,Todd,98.0,1
...,...,...,...,...
22364,371942068,Joy,209.0,30
22366,122959159,Jason,103.0,2
22368,9531956,Andrea,38.0,1
22375,132551861,Liberty,83.0,2


In [55]:
host_listing7.loc[host_listing7['host_id'] == 111808435]

,id,host_id,host_name,host_response_rate,review_scores_rating,number_of_reviews,host_is_superhost,last_review,property_type,room_type,price,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,listing_url,overall_rating,Host_List_count,Avg_Price


---
### Top 10 Superhost

In [56]:
True_listing = pd.concat(dup_host)
True_listing = True_listing.sort_values(by = ["Host_List_count"],ascending=False)
Top_50 = True_listing.drop_duplicates(subset=['host_id'])
Top_50.head(10)
#True_listing["Host_List_count"].isnull().any()

,id,host_id,host_name,host_response_rate,review_scores_rating,number_of_reviews,host_is_superhost,last_review,property_type,room_type,price,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,listing_url,overall_rating,Host_List_count,Avg_Price
2272,9754839,39073224,Maui Resort Rentals,100,5.00,3,t,2019-09-28,Entire condominium (condo),Entire home/apt,565.0,Lahaina,Maui,20.94384,-156.69079,https://www.airbnb.com/rooms/9754839,4.88,169,636.82
15297,43628194,22635145,CB Island Vacations,100,5.00,6,t,2021-09-11,Entire townhouse,Entire home/apt,150.0,North Kona,Hawaii,19.57472,-155.96488,https://www.airbnb.com/rooms/43628194,4.86,161,314.53
4076,16222147,96255708,Jeff And Suzanne,95,4.88,8,t,2021-04-02,Entire townhouse,Entire home/apt,352.0,South Kohala,Hawaii,19.93871,-155.85950,https://www.airbnb.com/rooms/16222147,4.89,88,474.51
2011,8581598,41842769,Resortica,100,4.82,28,t,2021-07-09,Entire townhouse,Entire home/apt,412.0,South Kohala,Hawaii,19.91521,-155.88067,https://www.airbnb.com/rooms/8581598,4.83,68,260.78
16673,46266538,57940735,Pmi,100,5.00,11,t,2021-09-02,Entire condominium (condo),Entire home/apt,266.0,Kihei-Makena,Maui,20.71392,-156.44238,https://www.airbnb.com/rooms/46266538,4.85,65,308.02
15422,43729487,43068394,Vacation-Maui,100,4.92,12,t,2021-08-22,Entire condominium (condo),Entire home/apt,207.0,Kihei-Makena,Maui,20.71318,-156.44191,https://www.airbnb.com/rooms/43729487,4.88,47,805.98
18685,48487428,388059369,Shelly,98,5.00,3,t,2021-07-08,Entire condominium (condo),Entire home/apt,444.0,Lahaina,Maui,20.97008,-156.67971,https://www.airbnb.com/rooms/48487428,4.87,45,393.67
6921,22371850,30787515,Brooke,99,4.80,5,t,2021-08-05,Entire condominium (condo),Entire home/apt,254.0,North Kona,Hawaii,19.62014,-155.98585,https://www.airbnb.com/rooms/22371850,4.80,45,243.20
18750,48571230,20704144,William,100,5.00,6,t,2021-09-01,Entire serviced apartment,Entire home/apt,176.0,North Kona,Hawaii,19.57031,-155.96431,https://www.airbnb.com/rooms/48571230,4.87,45,179.11
17705,47582788,382903817,Cimi,100,4.92,24,t,2021-09-10,Entire condominium (condo),Entire home/apt,279.0,Lahaina,Maui,20.99287,-156.66789,https://www.airbnb.com/rooms/47582788,4.80,42,315.50


---

In [57]:
Top_50.to_excel('Top_10.xlsx',sheet_name='Sheet1')